# Gradient Experiment: 
### pass 1 validation image (imagenet) through VGG and backpropagate the *correct* label.
### Then 

In [19]:
import sys
sys.path.append('../')

import numpy as np
from scipy import ndimage

from skimage.filters import sobel_h
from skimage.filters import sobel_v
from scipy import stats


import os
import matplotlib
import matplotlib.pyplot as plt
import scienceplots
from tensorflow.python.client import device_lib

#plt.rcParams['figure.figsize'] = [10,10]

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications import VGG16

from tensorflow.nn import depthwise_conv2d
from tensorflow.math import multiply, reduce_sum, reduce_mean,reduce_euclidean_norm, sin, cos, abs
from tensorflow import stack, concat, expand_dims

import tensorflow_probability as tfp

from utils.utils import *


plt.style.use(['science', 'ieee'])
plt.rcParams.update({'figure.dpi': '100'})

#### Experiment #1 : $\beta$ accross all layers (Top-10% filters)

In [20]:
model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [21]:
k = 10 # Top 10% highest l2-magnitude filters

In [41]:
conv_layers = []
for l in model.layers:
    if 'conv2d' in str(type(l)).lower():
        conv_layers.append(l)
e_beta = []
beta = []
min_beta = []
max_beta = []
for l_num, l in enumerate(conv_layers):
    print(f" ============ LAYER {l_num} ==================")
    top10F=topKfilters(model, l_num, k=k)
    beta_f = []
    for f in top10F:
        top10C = topKchannels(model, l_num, f, k=k)

        filters = tf.expand_dims(get_filter(model, l_num)[:,:,top10C, f], -1)
        s, a = getSymAntiSymTF(filters)
        a_energy = reduce_euclidean_norm(a, axis=[0,1])**2
        s_energy = reduce_euclidean_norm(s, axis=[0,1])**2
        total_energy  = reduce_euclidean_norm(filters, axis=[0,1])**2
        #beta = a_energy/total_energy
        beta_f.append(tf.reduce_mean(a_energy/total_energy))
    beta.append(beta_f)
    print(beta)
    e_beta.append(np.mean(beta[-1]))
    '''min_beta.append(e_beta[-1] - np.min(beta))
    max_beta.append(np.max(beta)-e_beta[-1])'''




 ============ LAYER 0 ==================
[[<tf.Tensor: shape=(), dtype=float32, numpy=0.66356236>, <tf.Tensor: shape=(), dtype=float32, numpy=0.9087973>, <tf.Tensor: shape=(), dtype=float32, numpy=0.91104394>, <tf.Tensor: shape=(), dtype=float32, numpy=0.9969793>, <tf.Tensor: shape=(), dtype=float32, numpy=0.9997209>, <tf.Tensor: shape=(), dtype=float32, numpy=0.8681362>]]
 ============ LAYER 1 ==================
[[<tf.Tensor: shape=(), dtype=float32, numpy=0.66356236>, <tf.Tensor: shape=(), dtype=float32, numpy=0.9087973>, <tf.Tensor: shape=(), dtype=float32, numpy=0.91104394>, <tf.Tensor: shape=(), dtype=float32, numpy=0.9969793>, <tf.Tensor: shape=(), dtype=float32, numpy=0.9997209>, <tf.Tensor: shape=(), dtype=float32, numpy=0.8681362>], [<tf.Tensor: shape=(), dtype=float32, numpy=0.0035362274>, <tf.Tensor: shape=(), dtype=float32, numpy=0.540005>, <tf.Tensor: shape=(), dtype=float32, numpy=0.5546729>, <tf.Tensor: shape=(), dtype=float32, numpy=0.8592329>, <tf.Tensor: shape=(), dty